In [5]:
import numpy as np
import matplotlib.pyplot as plt
import os

In [197]:
def make_shape(shapename,filename=None,duplicate=False,filename2=None):
    if shapename == 'circle':
        pm = np.array([1.,-1.])
        cen_scale = 5. #sets range of acceptable centers (make sure this keeps shape within axes)
        x = cen_scale*np.random.random()*np.random.choice(pm) #generate random x/y centers
        y = cen_scale*np.random.random()*np.random.choice(pm)
        rad_max = 4. #max radius minus one
        rad_min = 1. #sets minimum radius
        radius = (rad_max*np.random.random()) + rad_min #generate random radius
        color = (np.random.random(),np.random.random(),np.random.random())
        alpha = np.random.random()
        while alpha < 0.1:
            alpha = np.random.random()
        shape = plt.Circle((x,y),radius,color=color,alpha=alpha)
        if duplicate == True:
            alpha2 = np.random.random()
            while alpha2 < 0.1:
                alpha2 = np.random.random()
            shape2 = plt.Circle((x,y),radius,color=color,alpha=alpha2)
    elif shapename == 'square':
        #x,y = lower left corner of rectangle
        #angle = rotation angle of square 
        sidemin = 1.
        sidescale = 5.
        side = sidescale*np.random.random() + sidemin #randomly generate length of sides
        #can add side2 for rectangular shapes
        cen_offset = side/2. #offset for center coord.s because x,y are lower left corner coords for rectangle
        pm = np.array([1.,-1.])
        cen_scale = 3. #sets range of acceptable centers (make sure this keeps shape within axes)
        x = cen_scale*np.random.random()*np.random.choice(pm) - cen_offset #generate random x/y centers
        y = cen_scale*np.random.random()*np.random.choice(pm) - cen_offset
        angle = 360 * np.random.random()
        color = (np.random.random(),np.random.random(),np.random.random())
        alpha = np.random.random()
        while alpha < 0.1:
            alpha = np.random.random()
        shape = plt.Rectangle((x,y),height=side,width=side,angle=angle,color=color,alpha=alpha)
        if duplicate == True:
            alpha2 = np.random.random()
            while alpha2 < 0.1:
                alpha2 = np.random.random()
            shape2 = plt.Rectangle((x,y),height=side,width=side,angle=angle,color=color,alpha=alpha2)
    
    fig,ax = plt.subplots(edgecolor=None,figsize=(4,4))
    ax.add_artist(shape)
    ax.set_aspect('equal')
    ax.set_xlim(-12,12)
    ax.set_ylim(-12,12)
    plt.axis('off')
    plt.show()
    if filename is not None:
        plt.savefig(filename)
    plt.close(fig)
    
    if duplicate == True:
        fig,ax = plt.subplots(edgecolor=None,figsize=(4,4))
        ax.add_artist(shape2)
        ax.set_aspect('equal')
        ax.set_xlim(-12,12)
        ax.set_ylim(-12,12)
        plt.axis('off')
        if filename2 is not None:
            plt.savefig(filename2)
        plt.close(fig)
    return
        
def createCircularMask(h, w, center=None, radius=None):
    #from stack overflow: https://stackoverflow.com/questions/44865023/circular-masking-an-image-in-python-using-numpy-arrays
    if center is None: # use the middle of the image
        center = [int(w/2), int(h/2)]
    if radius is None: # use the smallest distance between the center and image walls
        radius = min(center[0], center[1], w-center[0], h-center[1])

    Y, X = np.ogrid[:h, :w]
    dist_from_center = np.sqrt((X - center[0])**2 + (Y-center[1])**2)

    mask = dist_from_center <= radius
    return mask

def createRectangularMask(h, w, center):
    orientation = np.random.choice(['vert','horiz'])
    if orientation == 'vert':
        height_max = min(center[1],h-center[1])
        height_min = 50. #pixels
        height = (height_max*np.random.random()) + height_min
        if height > height_max:
            height = height_max
        width_max = 35.
        width = (width_max*np.random.random()) + 1.

    elif orientation == 'horiz':
        width_max = min(center[0],w-center[0])
        width_min = 50 #pixels
        width = (width_max*np.random.random()) + width_min
        if width > width_max:
            width = width_max
        height_max = 35. #pixels
        height = (height_max*np.random.random()) +1.
    wmin = center[0]-width
    wmax = center[0]+width
    hmin = center[1]-height
    hmax = center[1]+height
#    mask = [wmin:wmax, hmin:hmax]
    mask = 0
    return mask

def make_shape2(h,w,shape,filename,duplicate=False,filename2=None):
    im_array = np.zeros((h,w))
    center0 = [int(w/2), int(h/2)]
    pm = np.array([1.,-1.])
    cen_scale = 50. #sets range of acceptable centers, in pixels from true center
    x = cen_scale*np.random.random()*np.random.choice(pm) #generate random x/y centers
    y = cen_scale*np.random.random()*np.random.choice(pm)
    cen_offset = [x, y]
    center = [center0[0]+x, center0[1]+y]
    brightness = 255.*np.random.random() #set shape brightness. 0=black,255=white
    while brightness < 50:
        brightness = 255.*np.random.random()
    if shape == 'circle':
        radius_max = min(center[0], center[1], w-center[0], h-center[1])
        radius_min = 10. #pixels
        radius = (radius_max*np.random.random()) + radius_min #generate random radius, keeping full circle in image
        if radius > radius_max:
            radius = radius_max
        mask = createCircularMask(h,w,center,radius)
        im_array[mask] = brightness
        if duplicate == True:
            im_array2 = np.zeros((h,w))
            brightness2 = 255.*np.random.random()
            while brightness2 < 50:
                brightness2 = 255.*np.random.random()
            im_array2[mask] = brightness2
    elif shape == 'rectangle':
        orientation = np.random.choice(['vert','horiz'])
        if orientation == 'vert':
            height_max = min(center[1],h-center[1])
            height_min = 50. #pixels
            height = (height_max*np.random.random()) + height_min
            if height > height_max:
                height = height_max
            width_max = 35.
            width = (width_max*np.random.random()) + 1.
        elif orientation == 'horiz':
            width_max = min(center[0],w-center[0])
            width_min = 50 #pixels
            width = (width_max*np.random.random()) + width_min
            if width > width_max:
                width = width_max
            height_max = 35. #pixels
            height = (height_max*np.random.random()) +1.
        wmin = int(center[0]-width)
        wmax = int(center[0]+width)
        hmin = int(center[1]-height)
        hmax = int(center[1]+height)
        im_array[wmin:wmax, hmin:hmax] = brightness
        if duplicate == True:
            im_array2 = np.zeros((h,w))
            brightness2 = 255.*np.random.random()
            while brightness2 < 50:
                brightness2 = 255.*np.random.random()
            im_array2[wmin:wmax, hmin:hmax] = brightness2
    
    plt.style.use('dark_background')
    fig,ax = plt.subplots(edgecolor=None,figsize=(4,4))
    ax.imshow(im_array,'gray',origin='lower',vmin=0,vmax=255,interpolation=None)
    ax.set_aspect('equal')
    #ax.set_xlim(-12,12)
    #ax.set_ylim(-12,12)
    plt.axis('off')
    if filename is not None:
        plt.savefig(filename)
        #plt.show()
    plt.close(fig)
    
    if duplicate == True:
        fig2,ax2 = plt.subplots(edgecolor=None,figsize=(4,4))
        ax2.imshow(im_array2,'gray',origin='lower',vmin=0,vmax=255,interpolation=None)
        ax2.set_aspect('equal')
        plt.axis('off')
        if filename2 is not None:
            plt.savefig(filename2)
        plt.close(fig2)
    return


In [198]:
make_shape2(256,256,shape='rectangle',filename='testrect.jpg')

In [201]:
nfigs = 500
outdir = 'testset_bw/rectangle/'
for i in range(nfigs):
    f = 'rect_'+str(i)+'.jpg'
    filename = os.path.join(outdir,f)
    make_shape2(256,256,'rectangle',filename)

In [202]:
nfigs = 100
outdir = os.path.join('predictset_bw','images')
dbls = [4,8,18,23,40,51,61,78,85,95]
for i in range(nfigs):
    options = ['circle','rectangle']
    choice = np.random.choice(options)
    f = 'image_'+str(i)+choice+'.jpg'
    filename = os.path.join(outdir,f)
    if i in dbls:
        f2 = 'image_'+str(i)+choice+'2.jpg'
        filename2 = os.path.join(outdir,f2)
        make_shape2(256,256,choice,filename,duplicate=True,filename2=filename2)
    else:
        make_shape2(256,256,choice,filename)

In [78]:
np.random.randint(100)

79